In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score
import requests
import zipfile
import os

In [3]:
# Download and unzip the dataset
data_path = "./data/"

# Load the data with proper dtype handling
books = pd.read_csv(
    data_path + "BX-Books.csv",
    sep=",",
    header=0,
    index_col=0,
    on_bad_lines="skip",
    encoding="latin-1",
    low_memory=False,  # Option to avoid DtypeWarning
)

users = pd.read_csv(
    data_path + "BX-Users.csv",
    sep=",",
    header=0,
    index_col=0,
    on_bad_lines="skip",
    encoding="latin-1",
    low_memory=False,  # Option to avoid DtypeWarning
)

ratings = pd.read_csv(
    data_path + "BX-Book-Ratings.csv",
    sep=",",
    header=0,
    index_col=0,
    on_bad_lines="skip",
    encoding="latin-1",
    low_memory=False,  # Option to avoid DtypeWarning
)

# Print the column names to check for 'ISBN'
print("Books columns:", books.columns)
print("Users columns:", users.columns)
print("Ratings columns:", ratings.columns)

Books columns: Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')
Users columns: Index(['User-ID', 'Location', 'Age'], dtype='object')
Ratings columns: Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object')


In [4]:
# Preprocess the data
ratings = ratings.merge(books[["ISBN", "Book-Title"]], on="ISBN")
ratings = ratings.drop(["ISBN"], axis=1)

In [5]:
# Filter books with at least min_book_ratings ratings
min_book_ratings = 10
book_rating_counts = ratings["Book-Title"].value_counts()
filtered_books = book_rating_counts[book_rating_counts >= min_book_ratings].index
ratings = ratings[ratings["Book-Title"].isin(filtered_books)]

# Filter users who rated at least min_user_ratings books
min_user_ratings = 5
user_rating_counts = ratings["User-ID"].value_counts()
filtered_users = user_rating_counts[user_rating_counts >= min_user_ratings].index
ratings = ratings[ratings["User-ID"].isin(filtered_users)]

# Create the user-item matrix
user_item_matrix = ratings.pivot_table(
    index="User-ID", columns="Book-Title", values="Book-Rating"
).fillna(0)

In [6]:
# Compute the cosine similarity matrix
cosine_sim_matrix = cosine_similarity(user_item_matrix)

In [7]:
# Split user indices into train and test sets
user_indices = np.arange(user_item_matrix.shape[0])
train_user_indices, test_user_indices = train_test_split(
    user_indices, test_size=0.2, random_state=42
)

In [8]:
def collaborative_filtering_recommendation(user_index, k=10):
    user_similarities = cosine_sim_matrix[user_index]
    top_k_similar_users = np.argsort(user_similarities)[-k - 1 : -1][::-1]
    similar_users_ratings = user_item_matrix.iloc[top_k_similar_users]
    mean_ratings = similar_users_ratings.mean(axis=0)
    top_k_books = mean_ratings.sort_values(ascending=False).head(k).index
    return top_k_books

In [9]:
def random_recommendation(ratings, n=10):
    unique_books = ratings["Book-Title"].unique()
    random_books = np.random.choice(unique_books, size=n, replace=False)
    return random_books


random_books = random_recommendation(ratings, n=10)
print("Random Score-Based Recommendations:")
for i, book in enumerate(random_books, 1):
    print(f"{i}. {book}")

Random Score-Based Recommendations:
1. Man from Mundania (Xanth Trilogy, No 12)
2. Dawn's Wicked Stepsister (Baby-Sitters Club, 31)
3. The Five Love Languages of Children
4. Diary of a Nobody
5. The Twisted Root
6. Lying in Wait: A J.P. Beaumont Mystery
7. Material Girl (Berkley Sensation)
8. Father'S Heart  (Family Man) (Superromance , No 786)
9. Macbeth (Dover Thrift Editions)
10. A Whole New Light


In [10]:
def popularity_recommendation(ratings, n=10):
    popular_books = (
        ratings.groupby("Book-Title")["Book-Rating"]
        .count()
        .sort_values(ascending=False)
        .head(n)
        .index
    )
    return popular_books


popular_books = popularity_recommendation(ratings, n=10)
print("\nPopularity-Based Recommendations:")
for i, book in enumerate(popular_books, 1):
    print(f"{i}. {book}")


Popularity-Based Recommendations:
1. Wild Animus
2. The Lovely Bones: A Novel
3. The Da Vinci Code
4. The Nanny Diaries: A Novel
5. Bridget Jones's Diary
6. A Painted House
7. The Secret Life of Bees
8. Divine Secrets of the Ya-Ya Sisterhood: A Novel
9. Angels &amp; Demons
10. Life of Pi


In [11]:
def recommend_books(user_index, strategy, k=10):
    if strategy == "collaborative":
        top_k_books = collaborative_filtering_recommendation(user_index, k=k)
    elif strategy == "random":
        top_k_books = random_recommendation(ratings, n=k)
    elif strategy == "popularity":
        top_k_books = popularity_recommendation(ratings, n=k)
    else:
        raise ValueError("Invalid recommendation strategy")
    return top_k_books

In [12]:
def evaluate_model(strategy, k=10):
    true_positive = 0
    false_positive = 0
    false_negative = 0

    for user_index in test_user_indices:
        true_books = set(
            user_item_matrix.iloc[user_index][
                user_item_matrix.iloc[user_index] > 0
            ].index
        )
        recommended_books = set(recommend_books(user_index, strategy, k))

        tp = len(true_books.intersection(recommended_books))
        fp = len(recommended_books - true_books)
        fn = len(true_books - recommended_books)

        true_positive += tp
        false_positive += fp
        false_negative += fn

    precision = true_positive / (true_positive + false_positive)
    recall = true_positive / (true_positive + false_negative)

    return precision, recall

In [13]:
# Evaluate the collaborative filtering model
cf_precision, cf_recall = evaluate_model(strategy="collaborative")
print(
    f"Collaborative Filtering: Precision = {cf_precision:.4f}, Recall = {cf_recall:.4f}"
)

Collaborative Filtering: Precision = 0.3110, Recall = 0.2601


In [14]:
# Evaluate the random score-based recommendation model
random_precision, random_recall = evaluate_model(strategy="random")
print(
    f"Random Score-Based: Precision = {random_precision:.4f}, Recall = {random_recall:.4f}"
)

Random Score-Based: Precision = 0.0009, Recall = 0.0007


In [15]:
# Evaluate the popularity-based recommendation model
popularity_precision, popularity_recall = evaluate_model(strategy="popularity")
print(
    f"Popularity-Based: Precision = {popularity_precision:.4f}, Recall = {popularity_recall:.4f}"
)

Popularity-Based: Precision = 0.0200, Recall = 0.0167
